# Create Merged Dataset

In this workbook, you will read in the `trip` and `fare` files. You are welcome to use DataFrame and/or SparkSQL API as you desire as long as it produces the expected results.

## Instructions:

1. Join both datasets such that you get a merged dataset with 21 unique fields. You need to determine how to join the dataset.
2. Once you create the merged dataset, you need to convert fields to the following types, since all fields were read is as string:
    * pickup_datetime and dropoff_datetime must be TIMESTAMP
    * passenger_count and rate_code must be INT
    * all other numeric fields must be FLOAT
    * the remaining fields stay as STRING
3. Save your merged and converted dataset to your own S3 bucket in parquet format.

You are welcome to add as many cells as you need below up until the next section. **You must include comments in your code.**

## First start my spark session

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()

In [2]:
spark

## Load and Explore data

In [3]:
trip = spark.read.parquet("s3://bigdatateaching/nyctaxi-2013/parquet/trip/")

In [4]:
fare = spark.read.parquet('s3://bigdatateaching/nyctaxi-2013/parquet/fare/')

In [5]:
trip.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [6]:
trip.count()

173179759

In [8]:
fare.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [9]:
fare.count()

173179759

In [7]:
# outer join those two dataset
# to prevent that some columns in trip can not be matched by the corresponding ones in fare
import pyspark.sql.functions as F
join_df = trip.join(fare, (trip.medallion == fare.medallion) & 
                          (trip.hack_license == fare.hack_license) & 
                          (trip.vendor_id == fare.vendor_id) &
                          (trip.pickup_datetime == fare.pickup_datetime),'outer').select(
                    F.coalesce(trip.medallion,fare.medallion).alias('medallion'),
                    F.coalesce(trip.hack_license,fare.hack_license).alias('hack_license'),
                    F.coalesce(trip.vendor_id,fare.vendor_id).alias('vendor_id'),
                    F.coalesce(trip.pickup_datetime,fare.pickup_datetime).alias('pickup_datetime'),
                    trip.rate_code, trip.store_and_fwd_flag, trip.dropoff_datetime, trip.passenger_count,
                    trip.trip_time_in_secs, trip.trip_distance, trip.pickup_longitude,
                    trip.pickup_latitude, trip.dropoff_longitude, trip.dropoff_latitude,
                    fare.payment_type, fare.fare_amount, fare.surcharge, fare.mta_tax,
                    fare.tip_amount, fare.tolls_amount, fare.total_amount)   

In [8]:
join_df

21

In [10]:
# change column types as requested
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import IntegerType

timestampcol = ['pickup_datetime','dropoff_datetime']
intcol = ['passenger_count','rate_code']
floatcol = ['trip_time_in_secs','trip_distance','fare_amount','surcharge',
            'mta_tax','tip_amount','tolls_amount','total_amount']

for col in timestampcol:
    join_df = join_df.withColumn(col, to_timestamp(col))

for col in intcol:
    join_df = join_df.withColumn(col,join_df[col].cast(IntegerType()))

for col in floatcol:
    join_df = join_df.withColumn(col,join_df[col].cast('float'))
    
#join_df = join_df.withColumn("pickup_datetime", to_timestamp('pickup_datetime')) \
#                 .withColumn("dropoff_datetime", to_timestamp('dropoff_datetime')) \
#                 .withColumn('passenger_count', join_df['passenger_count'].cast(IntegerType())) \
#                 .withColumn('rate_code', join_df['rate_code'].cast(IntegerType())) 

In [11]:
join_df.show(10)

+--------------------+--------------------+---------+-------------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|rate_code|store_and_fwd_flag|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+---------+-------------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|00005007A9F30E289...|02015F5B7D1884620...|

## In the following cells, please provide the requested code and output. Do not change the order and/or structure of the cells.

In the next cell, provide the code that saves your merged dataset to your S3 bucket.

In [16]:
join_df.write.parquet('s3a://jiaruxu-bigdatasummer/a4/merged_data')

In the next cell, print the schema of your merged dataset.

In [12]:
join_df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



In the next cell, print the first 10 records of your merged dataset.

In [13]:
join_df.show(10)

+--------------------+--------------------+---------+-------------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|rate_code|store_and_fwd_flag|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+---------+-------------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|00005007A9F30E289...|02015F5B7D1884620...|

In the next cell, print the row count of your merged dataset.

In [14]:
join_df.count()

173185091

In [17]:
spark.stop()